Load data

In [1]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader
from trainV1 import train, eval
from plot import *
from preprocessing import dataset_split_to_loader
from networks_lstm import PARC
import os

os.environ["CUDA_VISIBLE_DEVICES"]="2"
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()

In [2]:
# define arguments
class objectview(object):
    def __init__(self, d) -> None:
        self.__dict__ = d
    def setattr(self, attr_name, attr_value):
        self.__dict__[attr_name] = attr_value

args = objectview(d={
    'total_time': 4.8,
    'n_times': 201,
    'batch_size': 1000,
    'batch_n_times': 50,
    'batch_step': 1,
    'batch_recursive': True,
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'lr': 5e-7,
    'weight_decay': 5e-4,
    'epoch': 100,
    'criterion': torch.nn.MSELoss(),
    'n_hiddenfields': 64,
    'n_hiddens': 10
})

In [3]:
# Build a dataset
dataset = OneDDatasetBuilder(
    raw_dir='/data1/tam/datasets_231228',
    root_dir='/data1/tam/downloaded_datasets_WT',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type=torch.float32
)

Processing...


Process subject number 0, subject name : CEMENT416.
Process subject number 1, subject name : 10291.
Process subject number 2, subject name : CEMENT254.
Process subject number 3, subject name : 10296.
Process subject number 4, subject name : 10167.
Process subject number 5, subject name : 10134.
Process subject number 6, subject name : 10081.
Process subject number 7, subject name : 10088.
Process subject number 8, subject name : 10132.
Process subject number 9, subject name : CEMENT077.
Process subject number 10, subject name : CEMENT085.
Process subject number 11, subject name : CEMENT011.
Process subject number 12, subject name : 10133.
Process subject number 13, subject name : CEMENT220.
Process subject number 14, subject name : 10119.
Process subject number 15, subject name : CEMENT019.
Process subject number 16, subject name : 10323.
Process subject number 17, subject name : CEMENT256.
Process subject number 18, subject name : CEMENT021.
Process subject number 19, subject name : C

Done!


In [ ]:
# Load to normalize and batch
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normed_and_batched',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

# normalized_dataset = dataset.normalizing(
#     sub_dir='normalized',
#     scalers = {
#         'node_attr' : ['minmax_scaler', 0],
#         'edge_attr' : ['quantile_transformer', 0],
#         'pressure' : ['quantile_transformer', None],
#         'flowrate' : ['quantile_transformer', None]
#     }
# )

# batched_dataset = normalized_dataset.batching(
#     batch_size = args.batch_size,
#     batch_n_times = args.batch_n_times, 
#     recursive = args.batch_recursive, 
#     sub_dir='/normed_and_batched', 
#     step=args.batch_step
# )

Train

In [ ]:
## Prepare data
train_loader, test_loader = dataset_split_to_loader(
    dataset = dataset,
    subset_ids = {
        'train': list(range(0, 5)),
        'test': list(range(20, 25))
    },
    n_datas_per_batch = 10
)
train_loader, test_loader

In [ ]:
## Prepare model
_test_data = dataset[0]
model = PARC(
    n_fields=2,
    n_timesteps=_test_data.pressure.size(1),
    n_meshfields=(_test_data.node_attr.size(1),_test_data.edge_attr.size(1)), # Tuple(n_node_fields, n_mesh_fields)
    n_bcfields=1,
    n_hiddenfields=args.n_hiddenfields,
    n_hiddens=args.n_hiddens
).to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
args.setattr(attr_name='optimizer', attr_value=optimizer)

In [ ]:
model.derivative_solver.node_encoder

In [ ]:
# Training
total_train_loss = []
total_eval_loss = []
# batch = enumerate(list(range(0,10)))
for epoch in range(args.epoch):
    torch.cuda.empty_cache()
    train_loss = 0
    # for data in train_dataset:
    for i in range(train_loader.__len__()):
        data = next(iter(train_loader))
        train_loss += train(model=model, data=data, args=args)

    train_loss /= train_loader.__len__() # len(train_dataset)
    total_train_loss.append(train_loss)

    eval_loss = 0
    # for data in eval_dataset:
    for i in range(test_loader.__len__()):
        data = next(iter(test_loader))
        eval_loss += eval(model=model, data=data, args=args)
    eval_loss /= test_loader.__len__() #len(eval_dataset)
    total_eval_loss.append(eval_loss)
    
    print(f'Epoch {epoch}: train loss = {train_loss}; eval loss = {eval_loss}')
    if (epoch+1) % 25 == 0:
        torch.save(model.state_dict(), f'models/parc_v1_epoch{epoch+1}.pth')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_train_loss[:100])
plt.plot(total_eval_loss[:100])
# plt.ylim(0,10)
plt.show()

In [ ]:
# Load to evaluate
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    # sub_dir='normed_and_batched',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

data = dataset[0]
# args.device = torch.device('cpu')
model = PARC(
    n_fields=2,
    n_timesteps=data.pressure.size(1),
    n_meshfields=(data.node_attr.size(1),data.edge_attr.size(1)), # Tuple(n_node_fields, n_mesh_fields)
    n_bcfields=1,
    n_hiddenfields=args.n_hiddenfields,
    n_hiddens=args.n_hiddens
).to(args.device)
model.load_state_dict(torch.load(
    'models/parc_v1_epoch25.pth',
    map_location={'cuda:1':'cuda:0'}
))




## Number of timesteps of the current data (might be batched data)
model.n_timesteps = data.pressure.size(1)
timestep = args.total_time / model.n_timesteps

## Connectivity/edge_index: Tensor(2, n_edges)
edge_index = torch.cat([
    data.edge_index, 
    torch.flip(data.edge_index, dims=[0]
)], dim=1).to(args.device)

## Mesh features: Tuple(NodeTensor, EdgeTensor)
mesh_features = (
    data.node_attr.to(args.device).float(),            
    torch.cat([data.edge_attr.to(args.device).float()]*2,dim=0)
)

## Fields tensor(pressure, flowrate): Tensor(n_nodes, n_times, n_fields)
F_initial = torch.cat([
    data.pressure[:,0].unsqueeze(1), 
    data.flowrate[:,0].unsqueeze(1)
], dim=-1).to(args.device).float() # concat pressure and flowrate

## Boundary value tensor (flowrate at entrance): Tensor(n_nodes, n_times)
F_bc = torch.zeros((data.number_of_nodes, model.n_timesteps))
F_bc[0,:] = data.flowrate[0,:]
F_bc = F_bc.to(args.device).float()

## Predict output
with torch.no_grad():
    Fs, F_dots = model(
        F_initial=F_initial, 
        mesh_features=mesh_features, 
        edge_index=edge_index, 
        F_boundary=F_bc, 
        timestep=timestep
    )

    ## Ground truth Fields tensor(pressure, flowrate): Tensor(n_nodes, 1:n_times, n_fields)
    Fs_hat = torch.cat([
        data.pressure.unsqueeze(-1), 
        data.flowrate.unsqueeze(-1)
    ], dim=-1).to(args.device).float() # concat pressure and flowrate
    Fs_hat = Fs_hat[:,1:,:]

node_list = [10, 300, 5000]
## Draw pressure
for i_node in node_list:
    i_field = 0
    y_pred = Fs.cpu().numpy()[i_node,:,i_field]
    y_true = Fs_hat.cpu().numpy()[i_node,:,i_field]
    x = [i * 4.0 /200 for i in range(y_pred.shape[0])]
    # print(data.node_attr.numpy()[i_node, 6])
    # plt.ylim(-1,1)
    plt.plot(x, y_pred, c='red', label='GNN Euler')
    plt.plot(x, y_true, c='blue', linestyle='dashdot', label='ground_truth')
    # plt.ylim([-1,1])
    plt.legend(loc='upper right')
    plt.ylabel('Pressure', fontsize=20)
    plt.xlabel('Time', fontsize=20)
    plt.show()

## Draw flowrate
for i_node in node_list:
    i_field = 1
    y_pred = Fs.cpu().numpy()[i_node,:,i_field]
    y_true = Fs_hat.cpu().numpy()[i_node,:,i_field]
    x = [i * 4.0 /200 for i in range(y_pred.shape[0])]
    # print(data.node_attr.numpy()[i_node, 6])
    # plt.ylim(-1,1)
    plt.plot(x, y_pred, c='red', label='GNN Euler')
    plt.plot(x, y_true, c='blue', linestyle='dashdot', label='ground_truth')
    # plt.ylim([-1,1])
    plt.legend(loc='upper right')
    plt.ylabel('Flowrate', fontsize=20)
    plt.xlabel('Time', fontsize=20)
    plt.show()